# Hugging Face TRLを使用してLoRAアダプタでLLMを微調整する方法

このノートブックでは、LoRA（低ランク適応）アダプタを使用して大規模言語モデルを効率的に微調整する方法を示します。LoRAは、次のようなパラメータ効率の良い微調整技術です：
- 事前学習されたモデルの重みを固定
- 注意層に小さな学習可能なランク分解行列を追加
- 通常、学習可能なパラメータを約90％削減
- メモリ効率を維持しながらモデル性能を維持

以下の内容をカバーします：
1. 開発環境のセットアップとLoRA設定
2. アダプタトレーニング用のデータセットの作成と準備
3. `trl`と`SFTTrainer`を使用してLoRAアダプタで微調整
4. モデルのテストとアダプタの統合（オプション）


## 1. 開発環境のセットアップ

最初のステップは、Hugging FaceライブラリとPytorchをインストールすることです。`trl`、`transformers`、`datasets`を含みます。`trl`について聞いたことがない場合でも心配ありません。これは`transformers`と`datasets`の上に構築された新しいライブラリで、微調整、RLHF、オープンLLMの調整を容易にします。


In [ ]:
# Google Colabでの要件のインストール
# !pip install transformers datasets trl huggingface_hub

# Hugging Faceへの認証

from huggingface_hub import login

login()

# 便利のため、Hugging Faceのトークンを環境変数HF_TOKENとして設定できます

## 2. データセットの読み込み

In [13]:
# サンプルデータセットの読み込み
from datasets import load_dataset

# TODO: パスと名前のパラメータを使用してデータセットと設定を定義
dataset = load_dataset(path="HuggingFaceTB/smoltalk", name="everyday-conversations")
dataset

DatasetDict({
    train: Dataset({
        features: ['full_topic', 'messages'],
        num_rows: 2260
    })
    test: Dataset({
        features: ['full_topic', 'messages'],
        num_rows: 119
    })
})

## 3. `trl`と`SFTTrainer`を使用してLLMをLoRAで微調整

`trl`の[SFTTrainer](https://huggingface.co/docs/trl/sft_trainer)は、[PEFT](https://huggingface.co/docs/peft/en/index)ライブラリを通じてLoRAアダプタとの統合を提供します。このセットアップの主な利点は次のとおりです：

1. **メモリ効率**：
   - アダプタパラメータのみがGPUメモリに保存されます
   - ベースモデルの重みは固定され、低精度で読み込むことができます
   - 大規模モデルの消費者向けGPUでの微調整が可能

2. **トレーニング機能**：
   - 最小限のセットアップでネイティブPEFT/LoRA統合
   - さらにメモリ効率を向上させるためのQLoRA（量子化LoRA）サポート

3. **アダプタ管理**：
   - チェックポイント中のアダプタ重みの保存
   - ベースモデルにアダプタを統合する機能

例としてLoRAを使用します。これは、LoRAと4ビット量子化を組み合わせて、性能を犠牲にせずにメモリ使用量をさらに削減します。セットアップには次の手順が必要です：
1. LoRA設定（ランク、アルファ、ドロップアウト）を定義
2. PEFT設定でSFTTrainerを作成
3. アダプタ重みをトレーニングして保存


In [ ]:
# 必要なライブラリをインポート
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer, setup_chat_format
import torch

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)

# モデルとトークナイザーを読み込む
model_name = "HuggingFaceTB/SmolLM2-135M"

model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_name
).to(device)
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name)

# チャット形式を設定
model, tokenizer = setup_chat_format(model=model, tokenizer=tokenizer)

# 微調整の名前を設定
finetune_name = "SmolLM2-FT-MyDataset"
finetune_tags = ["smol-course", "module_1"]

`SFTTrainer`は、`peft`とのネイティブ統合をサポートしており、例えばLoRAを使用してLLMを効率的に微調整するのが非常に簡単です。`LoraConfig`を作成し、トレーナーに提供するだけです。

<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'>
    <h2 style='margin: 0;color:blue'>演習: 微調整のためのLoRAパラメータを定義</h2>
    <p>Hugging Faceのハブからデータセットを取得し、それを使用してモデルを微調整します。</p>
    <p><b>難易度レベル</b></p>
    <p>🐢 一般的なパラメータを使用して任意の微調整を行う</p>
    <p>🐕 パラメータを調整し、重みとバイアスでレビューする</p>
    <p>🦁 パラメータを調整し、推論結果の変化を示す</p>
</div>

In [ ]:
from peft import LoraConfig

# TODO: LoRAパラメータを設定
# r: LoRA更新行列のランク次元（小さいほど圧縮率が高い）
rank_dimension = 6
# lora_alpha: LoRA層のスケーリングファクター（高いほど適応が強い）
lora_alpha = 8
# lora_dropout: LoRA層のドロップアウト確率（過学習を防ぐのに役立つ）
lora_dropout = 0.05

peft_config = LoraConfig(
    r=rank_dimension,  # ランク次元 - 通常4-32の範囲
    lora_alpha=lora_alpha,  # LoRAスケーリングファクター - 通常ランクの2倍
    lora_dropout=lora_dropout,  # LoRA層のドロップアウト確率
    bias="none",  # LoRAのバイアスタイプ。対応するバイアスはトレーニング中に更新されます。
    target_modules="all-linear",  # LoRAを適用するモジュール
    task_type="CAUSAL_LM",  # モデルアーキテクチャのタスクタイプ
)

トレーニングを開始する前に、使用するハイパーパラメータ（`TrainingArguments`）を定義する必要があります。

In [ ]:
# トレーニング設定
# QLoRA論文の推奨に基づくハイパーパラメータ
args = SFTConfig(
    # 出力設定
    output_dir=finetune_name,  # モデルチェックポイントを保存するディレクトリ
    # トレーニング期間
    num_train_epochs=1,  # トレーニングエポック数
    # バッチサイズ設定
    per_device_train_batch_size=2,  # GPUごとのバッチサイズ
    gradient_accumulation_steps=2,  # 大きな効果的なバッチのための勾配蓄積
    # メモリ最適化
    gradient_checkpointing=True,  # メモリ節約のための計算トレードオフ
    # オプティマイザ設定
    optim="adamw_torch_fused",  # 効率のために融合されたAdamWを使用
    learning_rate=2e-4,  # 学習率（QLoRA論文）
    max_grad_norm=0.3,  # 勾配クリッピングの閾値
    # 学習率スケジュール
    warmup_ratio=0.03,  # ウォームアップのステップの割合
    lr_scheduler_type="constant",  # ウォームアップ後に学習率を一定に保つ
    # ロギングと保存
    logging_steps=10,  # Nステップごとにメトリックをログ
    save_strategy="epoch",  # 各エポックごとにチェックポイントを保存
    # 精度設定
    bf16=True,  # bfloat16精度を使用
    # 統合設定
    push_to_hub=False,  # HuggingFace Hubにプッシュしない
    report_to="none",  # 外部ロギングを無効化
)

すべてのビルディングブロックが揃ったので、`SFTTrainer`を作成してモデルのトレーニングを開始します。

In [ ]:
max_seq_length = 1512  # モデルとデータセットのパッキングの最大シーケンス長

# LoRA設定でSFTTrainerを作成
trainer = SFTTrainer(
    model=model,
    args=args,
    train_dataset=dataset["train"],
    peft_config=peft_config,  # LoRA設定
    max_seq_length=max_seq_length,  # 最大シーケンス長
    tokenizer=tokenizer,
    packing=True,  # 効率のために入力パッキングを有効化
    dataset_kwargs={
        "add_special_tokens": False,  # テンプレートで処理される特殊トークン
        "append_concat_token": False,  # 追加のセパレータは不要
    },
)

トレーニングを開始するには、`Trainer`インスタンスの`train()`メソッドを呼び出します。これにより、トレーニングループが開始され、モデルが3エポックにわたってトレーニングされます。PEFTメソッドを使用しているため、適応されたモデルの重みのみを保存し、完全なモデルは保存しません。

In [ ]:
# トレーニングを開始し、モデルは自動的にハブと出力ディレクトリに保存されます
trainer.train()

# モデルを保存
trainer.save_model()

  0%|          | 0/72 [00:00<?, ?it/s]

TrainOutput(global_step=72, training_loss=1.6402628521124523, metrics={'train_runtime': 195.2398, 'train_samples_per_second': 1.485, 'train_steps_per_second': 0.369, 'total_flos': 282267289092096.0, 'train_loss': 1.6402628521124523, 'epoch': 0.993103448275862})

Flash Attentionを使用して15kサンプルのデータセットで3エポックのトレーニングを行った結果、`g5.2xlarge`で4:14:36かかりました。このインスタンスのコストは`1.21$/h`で、合計コストは約`5.3$`です。



### LoRAアダプタを元のモデルに統合

LoRAを使用する場合、トレーニング中はアダプタ重みのみを学習し、ベースモデルは固定されます。トレーニング中は、これらの軽量なアダプタ重み（約2-10MB）のみを保存し、完全なモデルのコピーは保存しません。ただし、デプロイのためにアダプタをベースモデルに統合することを検討するかもしれません：

1. **デプロイの簡素化**：ベースモデル+アダプタの代わりに単一のモデルファイル
2. **推論速度**：アダプタ計算のオーバーヘッドなし
3. **フレームワークの互換性**：サービングフレームワークとの互換性向上


In [ ]:
from peft import AutoPeftModelForCausalLM


# CPU上でPEFTモデルを読み込む
model = AutoPeftModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=args.output_dir,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
)

# LoRAとベースモデルを統合して保存
merged_model = model.merge_and_unload()
merged_model.save_pretrained(
    args.output_dir, safe_serialization=True, max_shard_size="2GB"
)

## 3. モデルをテストして推論を実行

トレーニングが完了したら、モデルをテストしたいと思います。元のデータセットから異なるサンプルを読み込み、単純なループと精度をメトリックとして使用してモデルを評価します。



<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'>
    <h2 style='margin: 0;color:blue'>ボーナス演習: LoRAアダプタの読み込み</h2>
    <p>例のノートブックから学んだことを使用して、トレーニングされたLoRAアダプタを推論のために読み込みます。</p> 
</div>

In [30]:
# メモリを再度解放
del model
del trainer
torch.cuda.empty_cache()

In [ ]:
import torch
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer, pipeline

# PEFTアダプタでモデルを読み込む
tokenizer = AutoTokenizer.from_pretrained(finetune_name)
model = AutoPeftModelForCausalLM.from_pretrained(
    finetune_name, device_map="auto", torch_dtype=torch.float16
)
pipe = pipeline(
    "text-generation", model=merged_model, tokenizer=tokenizer, device=device
)

いくつかのプロンプトサンプルをテストし、モデルの性能を確認しましょう。

In [34]:
prompts = [
    "ドイツの首都はどこですか？その理由と過去に異なっていたかどうかを説明してください。",
    "数の階乗を計算するPython関数を書いてください。",
    "長さ25フィート、幅15フィートの長方形の庭があります。庭全体を囲むフェンスを作りたい場合、何フィートのフェンスが必要ですか？",
    "果物と野菜の違いは何ですか？それぞれの例を挙げてください。",
]


def test_inference(prompt):
    prompt = pipe.tokenizer.apply_chat_template(
        [{"role": "user", "content": prompt}],
        tokenize=False,
        add_generation_prompt=True,
    )
    outputs = pipe(
        prompt,
    )
    return outputs[0]["generated_text"][len(prompt) :].strip()


for prompt in prompts:
    print(f"    prompt:\n{prompt}")
    print(f"    response:\n{test_inference(prompt)}")
    print("-" * 50)